# Obteniendo datos de la API de Spotify

In [15]:
import base64
import requests
import pandas as pd
import os

## Autorización

In [25]:
client_id = os.environ['spotify_client_id']
client_secret = os.environ['spotify_client_secret']

In [26]:
# función de autenticación de spotify

def spoty_aut (client_id, client_secret):
    url = 'https://accounts.spotify.com/api/token'
    credenciales = client_id + ':' + client_secret
    authorization = "Basic " + base64.b64encode(credenciales.encode()).decode()
    grant_type= 'client_credentials'
    auth = requests.post(url, data={'grant_type':grant_type},
              headers={'Authorization':authorization, 'Content-Type':'application/x-www-form-urlencoded'})
    return auth.json()

In [27]:
auth=spoty_aut(client_id, client_secret)

## Obteniendo playlist

In [28]:
#función que crea df con variables necesarias para sherpa a partir de playlist_id

def df_sherpa(playlist_id):
    base = "https://api.spotify.com/v1/"
    playlist = requests.get(base+f"playlists/{playlist_id}/tracks",
                      headers = {'Authorization':f'''Bearer {auth['access_token']}''', 'Content-Type': 'application/json'})
    
    n_canciones=len(playlist.json()['items'])
    
    lista_df_tracks=[]
    
    for i in range(0,n_canciones):
        
        #obteniendo autor, nombre de canción y sus IDs de las canciones en la playlist
        artist_track=playlist.json()['items'][i]
        
        artist_id= artist_track['track']['artists'][0]['id']
        artist = artist_track['track']['artists'][0]['name']
        track = artist_track['track']['name']
        track_id = artist_track['track']['id']
        
        #obteniendo características de canciones de playlist
        audio_features = requests.get(base+f"audio-features/{track_id}",
                          headers = {'Authorization':f'''Bearer {auth['access_token']}''', 'Content-Type': 'application/json'})
        tempo = audio_features.json()['tempo']
        energy = audio_features.json()['energy']
        danceability = audio_features.json()['danceability']
        valence = audio_features.json()['valence']
        key = audio_features.json()['key']
        time_signature = audio_features.json()['time_signature']
        acousticness = audio_features.json()['acousticness']
        instrumentalness = audio_features.json()['instrumentalness']
        mode = audio_features.json()['mode']
    
        lista_track = [track_id, artist, track, tempo, energy, danceability, 
                 valence, key, time_signature, acousticness, instrumentalness, mode]
        lista_df_tracks.append(lista_track)
    
    df_sherpa=pd.DataFrame(lista_df_tracks, columns=["track_id", "artist", "track",  "tempo", "energy", "danceability", 
             "valence", "key", "time_signature", "acousticness", "instrumentalness", "mode"])
    return df_sherpa
    
    

In [29]:
#Aplicando la función para obtener variables de estudio de canciones de una playlist
df_test=df_sherpa("6doANzWu81vKWBsIiMLrHj")

In [31]:
# Exportando a CSV el df de la playlist
df_test.to_csv("/Users/santiago/ih-bootcamp/proyectos/sherpa/out/sherpa_test.csv")